## Code Instruction

Before running the code some keys need to be added to the **Secret Key** section as -

- NEO4J_URI="neo4j+s://7045c5e5.databases.neo4j.io"
- NEO4J_USERNAME="neo4j"
- NEO4J_PASSWORD="0yf9ICXY28oVEP-U2FYoFYgZ_jbjePPn1llGzvWOqok"
- AURA_INSTANCEID="7045c5e5"
- AURA_INSTANCENAME="TNBC"
- And the $Openai API Key$

In [1]:
from langchain.graphs import Neo4jGraph

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()


NEO4J_URI = os.getenv("server_NEO4J_URI")
NEO4J_USERNAME = os.getenv("server_NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("server_NEO4J_PASSWORD")
api_key = os.getenv('GROQ_API_KEY')
api_key_openai = os.getenv("OPENAI_API_KEY_99")


In [ ]:
# from neo4j import GraphDatabase

# url="bolt://localhost:7690"
# username="neo4j",
# password="12345678"

# AUTH = (username, password)

# with GraphDatabase.driver(url, auth = AUTH) as driver:
#     driver.verify_connectivity()

In [3]:
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [ ]:
## GROQ - LLM LOADING

from langchain_groq import ChatGroq


llm1 = ChatGroq (
    model= "mixtral-8x7b-32768",      # "mixtral-8x7b-32768" #"llama-3.1-70b-versatile", ## "llama-3.1-8b-instant"
    temperature=0.0,
    max_tokens=1500,
    timeout=None,
    max_retries=4,
    groq_api_key = api_key

)

In [4]:
## OPENAI - LLM LOADING

from langchain_openai import ChatOpenAI

llm2 = ChatOpenAI(
    model = "gpt-4o-mini",
    openai_api_key = api_key_openai,
    temperature = 0
)

In [6]:
llm2.invoke("Who is virat kohli?")

AIMessage(content="Virat Kohli is an Indian cricketer and one of the most prominent figures in international cricket. Born on November 5, 1988, in Delhi, India, he is known for his aggressive batting style, exceptional consistency, and leadership qualities. Kohli made his debut for the Indian national team in 2008 and quickly established himself as one of the top batsmen in the world.\n\nHe has served as the captain of the Indian cricket team in all three formats (Test, One Day Internationals, and T20s) and has led the team to numerous victories, including a historic Test series win in Australia. Kohli is also known for his records in various formats of the game, including being one of the fastest players to reach several milestones in terms of runs scored.\n\nIn addition to his international career, Kohli has played for the Royal Challengers Bangalore (RCB) in the Indian Premier League (IPL) and has been one of the league's standout performers. Off the field, he is known for his phila

In [8]:
# llm3 = ChatGroq (
#     model= "llama-3.1-70b-versatile",      # "mixtral-8x7b-32768" #"llama-3.1-70b-versatile", ## "llama-3.1-8b-instant"
#     temperature=0.0,
#     max_tokens=1500,
#     timeout=None,
#     max_retries=4,
#     groq_api_key = api_key

# )

In [7]:
# Match all modes in the graph

cypher = """
MATCH (n) RETURN count(n);
"""

graph.query(cypher)

[{'count(n)': 32166}]

In [39]:
from langchain.chains import GraphCypherQAChain

cypher_chain = GraphCypherQAChain.from_llm(
    llm = llm2,
    graph=graph,
    verbose=True,
    validate_cypher=True,
    return_intermediate_steps=True,
    top_k = 5000,
    allow_dangerous_requests=True
)

In [40]:
cypher_chain.invoke("What are the anatomy of the lung cancer ?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (a:Anatomy)<-[:anatomy_protein_present]-(g:Gene)<-[:disease_protein]-(d:Disease {name: 'lung cancer'})
RETURN a

Full Context:
[{'a': {'name': 'uterine cervix', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'Brodmann (1909) area 9', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'omental fat pad', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'anterior cingulate cortex', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'dorsolateral prefrontal cortex', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'adipose tissue of abdominal region', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'layer of synovial tissue', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'squamous epithelium', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'Brodmann (1909) area 46', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'nasal cavity epithelium', 'source': 'UBE

{'query': 'What are the anatomy of the lung cancer ?',
 'result': "I don't know the answer.",
 'intermediate_steps': [{'query': "\nMATCH (a:Anatomy)<-[:anatomy_protein_present]-(g:Gene)<-[:disease_protein]-(d:Disease {name: 'lung cancer'})\nRETURN a\n"},
  {'context': [{'a': {'name': 'uterine cervix',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'Brodmann (1909) area 9',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'omental fat pad', 'source': 'UBERON', 'type': 'anatomy'}},
    {'a': {'name': 'anterior cingulate cortex',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'dorsolateral prefrontal cortex',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'adipose tissue of abdominal region',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'layer of synovial tissue',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'squamous epithelium',
      's

In [10]:
# # Clear all nodes and relationships in the graph
# delete_query = "MATCH (n) DETACH DELETE n"
# graph.query(delete_query)

# print("Graph has been cleared of all nodes and relationships.")


## Vector Embedding

In [120]:
# !pip install langchain_nomic

In [5]:
# from langchain.embeddings import OllamaEmbeddings

# embeddings = OllamaEmbeddings(model = "llama3", show_progress=True)



from langchain_openai import OpenAIEmbeddings

embeddings_openai = OpenAIEmbeddings(model="text-embedding-ada-002", api_key = api_key_openai, show_progress_bar=True)




# from langchain_nomic import NomicEmbeddings

# embeddings = NomicEmbeddings(model = "nomic-embed-text-v1.5")


## Data Loading

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = CSVLoader("../Datasets/vertices.csv")
docu = loader.load()
len(docu)

24330

In [3]:
# Split data into documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=310, chunk_overlap=40)
docs = text_splitter.split_documents(docu)
docs[:10]

[Document(metadata={'source': '../Datasets/vertices.csv', 'row': 0}, page_content=': 3\nnode_name: NRF1\nnode_type: gene/protein\nnode_source: NCBI'),
 Document(metadata={'source': '../Datasets/vertices.csv', 'row': 1}, page_content=': 6\nnode_name: EIF3I\nnode_type: gene/protein\nnode_source: NCBI'),
 Document(metadata={'source': '../Datasets/vertices.csv', 'row': 2}, page_content=': 8\nnode_name: MT1A\nnode_type: gene/protein\nnode_source: NCBI'),
 Document(metadata={'source': '../Datasets/vertices.csv', 'row': 3}, page_content=': 11\nnode_name: RPS8\nnode_type: gene/protein\nnode_source: NCBI'),
 Document(metadata={'source': '../Datasets/vertices.csv', 'row': 4}, page_content=': 13\nnode_name: MRPL9\nnode_type: gene/protein\nnode_source: NCBI'),
 Document(metadata={'source': '../Datasets/vertices.csv', 'row': 5}, page_content=': 15\nnode_name: EGR1\nnode_type: gene/protein\nnode_source: NCBI'),
 Document(metadata={'source': '../Datasets/vertices.csv', 'row': 6}, page_content=': 18\n

In [124]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n" + d.page_content for i, d in enumerate(docs)]))

In [125]:
# docs

In [126]:
# vector_index = Neo4jVector.from_documents(
#     documents=docs,
#     embedding=embeddings,
#     search_type="hybrid",
#     node_label="Document", # Indicates that the nodes labeled "Document" in Neo4j will be indexed and used for search.
#     text_node_properties=["text"], # Specifies that the "text" property of the nodes will be used to generate embeddings.
#     embedding_node_property="embedding"
# )

# vector_index = Neo4jVector.from_documents(
#     documents=docs,
#     embedding=embeddings,
#     search_type="hybrid",  # Hybrid search
#     node_label="Document",  # Node label in Neo4j
#     embedding_node_property="embedding"
# )

# import time
# from neo4j.exceptions import ServiceUnavailable

# def retry_with_backoff(func, retries=5, backoff_in_seconds=2):
#     for i in range(retries):
#         try:
#             return func()
#         except ServiceUnavailable as e:
#             wait_time = backoff_in_seconds * (2 ** i)
#             print(f"Retrying in {wait_time} seconds due to: {e}")
#             time.sleep(wait_time)
#     raise Exception("Max retries exceeded")

# # Call the function that interacts with Neo4j
# vector_index = retry_with_backoff(lambda: Neo4jVector.from_documents(
#     documents=docs,
#     embedding=embeddings,
#     search_type="hybrid",
#     node_label="Document",
#     embedding_node_property="embedding"
# ))


In [127]:
# import time

# # Define the batch size for embedding
# batch_size = 2000

# # Function to embed a batch of documents and reconnect to Neo4j
# def embed_batch(batch_docs, batch_number):
#     # Reconnect to Neo4j Aura for each batch
#     Neo4jGraph(
#         url=NEO4J_URI,
#         username=NEO4J_USERNAME,
#         password=NEO4J_PASSWORD
#     )

#     print(f"Processing batch {batch_number} with {len(batch_docs)} documents...")

#     try:
#         # Perform vector embedding for the current batch
#         vector_index = Neo4jVector.from_documents(
#             documents=batch_docs,
#             embedding=embeddings,
#             search_type="hybrid",  # Hybrid search
#             node_label="Document",  # Node label in Neo4j
#             embedding_node_property="embedding"
#         )
#         print(f"Batch {batch_number} successfully embedded.")

#     except Exception as e:
#         print(f"Error embedding batch {batch_number}: {str(e)}")
#         time.sleep(10)  # Wait 10 seconds before retrying

# # Iterate through the documents in chunks of 2000
# for i in range(0, len(docs), batch_size):
#     batch_docs = docs[i:i+batch_size]  # Get the batch of documents
#     batch_number = i // batch_size + 1  # Track the batch number
#     embed_batch(batch_docs, batch_number)  # Embed the current batch
#     time.sleep(2)  # Optional: Pause between batches to reduce load

# print("All batches embedded successfully.")

# Graph retriever

## Vector Embedding Data Retrival

At first the whole corpus of data is, is embedded using the embedding technique (ollama/openai) to the graph database (neo4j), then the we use similarity search to retrive the  most prominent answer according to the query.

In [17]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_chroma import Chroma

In [9]:
# from langchain_groq import ChatGroq

# llm3 = ChatGroq (
#     model= "llama-3.1-70b-versatile",   # "llama-3.1-8b-instant", ## "llama-3.1-405b-reasoning" Not Working...
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     groq_api_key = api_key

# )

In [130]:
# vector_index = Neo4jVector(
#         graph=graph,
#         embedding=embeddings,
#         search_type="hybrid",  # Hybrid search
#         node_label="Document",  # Node label in Neo4j
#         embedding_node_property="embedding"
#     )

In [131]:
## the MultiQueryRetriever uses the LLM to interpret the query
# and generate several variations. This helps in covering different angles,
# synonyms, and related concepts that might be relevant
# but not explicitly mentioned in the original query.

# retriever = MultiQueryRetriever.from_llm(
#     vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 100}),
#     llm2,
#     prompt = QUERY_PROMPT
# )

# retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [132]:
# retrieved_docs = retriever.get_relevant_documents("liver carcinoma")
# l = []
# for i in retrieved_docs:
#   l.append(i.page_content)

# l

In [133]:
# x_names = []
# for entry in l:
#     for line in entry.split("\n"):
#         if "x_name" in line:
#             x_names.append(line.split(":")[1].strip())

# x_names

In [134]:
# # RAG prompt

# template = """
# You are an AI assistant tasked with answering questions based solely on the information retrieved from a vector database. The database stores context in an embedded format to ensure efficient similarity search, relevance, and accuracy. Your goal is to provide precise answers using only the provided context, with no external knowledge or assumptions.

# Follow these guidelines when answering:
# 1. If the context contains the information, provide a clear and exhaustive response.
# 2. If the context partially addresses the question, acknowledge the limitations and answer accordingly.
# 3. If the context doesn't contain the answer, explicitly state that the information is not available in the current context.
# 4. If there are multiple answers for a question, return **all** the answers.
# 5. For example, if querying about genes related to a disease, **fetch every gene** mentioned, not just a few. Ensure that all relevant entities are included, even if they are numerous.
# 6. If the question implies listing items, like genes, pathways, or drugs, ensure that the response is complete and includes everything found in the context.

# Context:
# {context}

# Question:
# {question}

# ### Examples:
# ---

# **Example 1:**

# Context:
# "The study identified several proteins involved in cancer progression, specifically focusing on p53, which acts as a tumor suppressor. Additionally, certain gene mutations like BRCA1 and BRCA2 were found to be crucial in triple negative breast carcinoma or tnbc . "

# Question:
# "What are the key genes involved in tnbc ?"

# Answer:
# "According to the context, the key genes involved in triple negative breast carcinoma (tnbc) are BRCA1 and BRCA2."

# ---

# **Example 2:**

# Context:
# "The database contains structural information about drug interactions, focusing on targeted therapies for breast cancer. However, no specific mention of immune therapies or general chemotherapy is made in this context."

# Question:
# "What types of therapies are mentioned for breast cancer?"

# Answer:
# "Based on the context, the therapies mentioned for breast cancer are targeted therapies. There is no mention of immune therapies or general chemotherapy."

# ---

# **Example 3:**

# Context:
# "No information is available regarding the use of CRISPR technology in cancer treatments in this part of the database."

# Question:
# "Is there any information about CRISPR technology in cancer treatments?"

# Answer:
# "There is no information about CRISPR technology in cancer treatments available in the current context."

# ---

# **Example 4:**

# Context:
# "Research highlights how hormone receptor status influences treatment options for breast cancer. For hormone-positive breast cancer, therapies often include estrogen blockers or hormone therapy, whereas hormone negative breast cancers require alternative treatments."

# Question:
# "What treatments are suggested for hormone negative breast cancer?"

# Answer:
# "The context indicates that hormone negative breast cancers require alternative treatments, though the specific alternatives are not mentioned."

# ---

# **Example 5:**

# Context:
# "The database includes clinical trial data for several drugs. However, it does not mention results related to the survival rates for patients undergoing combination therapies."

# Question:
# "Does the context mention survival rates for combination therapies?"

# Answer:
# "No, the context does not provide information about survival rates for combination therapies."

# ---
# """

# prompt = ChatPromptTemplate.from_template(template)


In [135]:
# vector_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | llm2
#     | StrOutputParser()
# )

In [136]:
# ans = vector_chain.invoke("Tell me all the genes of liver cancer")
# ans

In [137]:
# vector_chain.invoke("What are the different phenotypes present for the disease female breast carcinoma ?")

In [138]:
# vector_chain.invoke("What are the different genes responsible for the disease triple negative breast carcinoma ?")

## Graph Retriver

On the other hand, configuring a graph retrieval is more involved but offers more freedom. In this example, we will use a full-text index to identify relevant nodes and then return their direct neighborhood. We can easily find any node and accordingly we can find the other relationships.

In [139]:
# from langchain_core.pydantic_v1 import BaseModel, Field
# from typing import Tuple, List, Optional
# from langchain.prompts import ChatPromptTemplate

In [140]:
# ## ENTITY EXTRACTION

# graph.query(
#     "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]"
#     )


# # Extract entities from text
# class Entities(BaseModel):
#     """Identifying information about entities."""

#     names: List[str] = Field(
#         ...,
#         description = "All the diseases, drugs, genes or proteins, exposures, molecular_function, pathways, "
#         "phenotypes, cellular components, biological processes entities that appear in the text .",

#     )

# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are extracting diseases, drugs, genes or proteins, exposures, molecular_function, pathways, "
#             "phenotypes, cellular components, biological processes entities from the text ."
#             "You are extracting multi-word entities (like pathways or gene names) from the text. "
#             "Ensure to capture complete and meaningful phrases, and avoid partial entity identification. "
#             "Do semantic searching to understand context and relationships."
#         ),
#         (
#             "human",
#             "Use the given format to extract information from the following "
#             "input: {question}",
#         ),
#     ]
# )

# entity_chain = prompt | llm2.with_structured_output(Entities)

In [141]:
# # from langchain_nomic import NomicEmbeddings
# from langchain.vectorstores import Chroma

# path = "../vector_db"
# vector_db = Chroma.from_documents(
#     documents = docs,
#     embedding = embeddings_openai,
#     collection_name="primekg_new",
#     persist_directory=path
# )

Support for third party widgets will remain active for the duration of the session. To disable support:

Support for third party widgets will remain active for the duration of the session. To disable support:

In [142]:
# vector_db = Chroma(
#     embedding_function=embeddings_openai,
#     persist_directory="."

# )

# client = vector_db._client  # Accessing the Chroma client directly

# # Get all the collections
# collections = client.list_collections()

# # Delete all collections
# for collection in collections:
#     client.delete_collection(collection.name)


In [143]:
# vector_db = Chroma(
#     embedding_function=embeddings_openai,
#     persist_directory=".",
#     collection_name="primekg_new"
# )

In [144]:
# def pretty_print_docs(docs):
#     print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n" + d.page_content for i, d in enumerate(docs)]))

In [145]:
# retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [146]:
# from langchain_core.prompts.prompt import PromptTemplate
# from langchain.chains import LLMChain

# QUERY_PROMPT = PromptTemplate(
#     input_variables=["question"],
#     template="""
#     You are an AI language model specializing in information retrieval.

#     - Your task is to rephrase the user's question in five distinct and meaningful ways, ensuring that the question should be simple and focus solely on the meaning of the
#       main question.
#     - The five questions should semantically same but they are of differnt form. As a human can ask a same question in many ways.

#     Please ensure that each version is unique in phrasing, and explores alternative ways of framing the question while preserving the semantic meaning.


#     Do not use any synonyms of certain keyword present in user query. in user query you just need to rephrase, not use different synonyms of any word.

#     Lastly return all the differnt phrases in a list.

#     Original question: {question}

#     1.
#     2.
#     3.
#     4.
#     5.
#     """
# )

In [147]:
# retriever = MultiQueryRetriever.from_llm(
#     vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 10}),
#     llm2,
#     prompt = QUERY_PROMPT
# )

# retriever.get_relevant_documents("What are the genes for the activated for KRAS ?")

In [148]:
# def retrive_similar(ques):
#     l = []
#     x_names = []
#     ent = entity_chain.invoke(ques)
#     entity = ent.names[0]
#     retrieved_docs = retriever.get_relevant_documents(entity)
#     for i in retrieved_docs:
#       l.append(i.page_content)

#     for entry in l:
#         for line in entry.split("\n"):
#             if "node_name" in line:
#                 x_names.append(line.split(":")[1].strip())

#     if entity in x_names:
#       return ent.names

#     else:
#       return x_names[0]

# # Example use of an LLMChain to rephrase the question
# def rephrasing(ques):
#     llm_chain = LLMChain(
#         prompt=QUERY_PROMPT,
#         llm=llm2
#     )

#     response = llm_chain.run(ques)

#     l = response.strip().split('\n')

#     qq = []
#     for i in l:
#       qq.append(i[3:])

#     sim = retrive_similar(ques)
#     ent = entity_chain.invoke(ques)
#     entity = ent.names[0]
#     k = []
#     for i in qq:
#       k.append(i.replace(entity, sim[0]))

#     return k

### retrieval component of the RAG

**The structured_retriever** function starts by detecting entities in the user question. Next, it iterates over the detected entities and uses a Cypher template to retrieve the neighborhood of relevant nodes.

In [149]:
# # Fulltext index query
# def structured_retriever(question: str) -> str:
#     """
#     Collects the neighborhood of entities mentioned
#     in the question
#     """
#     result = ""
#     entities = retrive_similar({"question": question})
#     for entity in entities:
#         response = graph.query(
#             """
#             CALL db.index.fulltext.queryNodes('entity', $query, {limit: 2})
#             YIELD node, score
#             // Get all outgoing and incoming relationships from the node
#             MATCH (node)-[r:!MENTIONS]->(neighbor)
#             RETURN node.name + ' - ' + type(r) + ' -> ' + neighbor.name AS output
#             UNION
#             MATCH (node)<-[r]-(neighbor)
#             RETURN neighbor.name + ' - ' + type(r) + ' -> ' + node.name AS output
#             LIMIT 100
#             """,
#             {"query": entity}  # Adjust based on how you handle entity queries
#         )
#         result += "\n".join([el['output'] for el in response])
#     return result



In [150]:
# structured_retriever(ques)

In [151]:
# # Fulltext index query
# def structured_retriever(question: str) -> str:
#     """
#     Collects the neighborhood of entities mentioned
#     in the question
#     """
#     result = ""
#     entities = entity_chain.invoke({"question": question})
#     for entity in entities.names:
#         response = graph.query(
#             """
#             MATCH (node {name: $query})
#             MATCH (node)-[r]->(neighbor)
#             RETURN node.name + ' - ' + type(r) + ' -> ' + neighbor.name AS output
#             UNION
#             MATCH (node)<-[r]-(neighbor)
#             RETURN neighbor.name + ' - ' + type(r) + ' -> ' + node.name AS output
#             LIMIT 50
#             """,
#             {"query": entity}  # Direct name match
#         )
#         result += "\n".join([el['output'] for el in response])
#     return result



In [152]:
# def structured_retriever(question: str) -> str:
#     """
#     Retrieves neighborhood of disease-related entities, expanding up to k hops,
#     focusing on disease relationships first, with a maximum of 100 results.
#     Removes duplicate relationships and formats them.
#     """
#     result = ""
#     sim = retrive_similar(question)
#     for entity in sim:
#         response = graph.query(
#         """
#         MATCH (disease {name: $query, type: 'disease'})
#         CALL apoc.path.expandConfig(disease, {
#             relationshipFilter: '>',  // Direction of relationships (outgoing only)
#             maxLevel: 2,
#             limit: 750,
#             bfs: true,
#             uniqueness: 'NODE_GLOBAL'
#         })
#         YIELD path
#         RETURN
#             [node IN nodes(path) | node.name] AS path_nodes,
#             [rel IN relationships(path) | type(rel)] AS rel_types
#         ORDER BY length(path) ASC  // Order by shortest path first (BFS)

#         """,
#         {"query": entity}

#         )

#         # Remove duplicates
#         unique_response = []
#         seen = set()
#         for entry in response:
#             # Convert the list of nodes and relationships to tuples for comparison
#             entry_tuple = (tuple(entry['path_nodes']), tuple(entry['rel_types']))
#             if entry_tuple not in seen:
#                 seen.add(entry_tuple)
#                 unique_response.append(entry)

#         # Format the output as "entity1 - relationship -> entity2"
#         formatted_relations = []
#         for entry in unique_response:
#             path_nodes = entry['path_nodes']
#             rel_types = entry['rel_types']

#             if len(rel_types) > 0:
#                 for i in range(1, len(path_nodes)):
#                     formatted_relations.append(f"{path_nodes[i-1]} - {rel_types[i-1]} -> {path_nodes[i]}")

#         result += "\n".join(formatted_relations) + "\n"

#     return result

In [153]:
# ques = "Tell me all the drugs associated with lung carcinoma ."

In [154]:
# response = structured_retriever(ques)

In [155]:
# response

In [ ]:
# response = graph.query(
#         """
#         MATCH (disease {name: $query, type: 'disease'})
#         CALL apoc.path.expandConfig(disease, {
#             relationshipFilter: '>',  // Direction of relationships (outgoing only)
#             maxLevel: 2,
#             bfs: true,
#             uniqueness: 'NODE_GLOBAL'
#         })
#         YIELD path
#         RETURN
#             [node IN nodes(path) | node.name] AS path_nodes,
#             [rel IN relationships(path) | type(rel)] AS rel_types
#         ORDER BY length(path) ASC  // Order by shortest path first (BFS)

#         """,
#         {"query": "triple negative breast carcinoma"}

#         )

In [ ]:
# response

In [ ]:
# graph.query(
#             """
#             MATCH (node {name: $query})
#             MATCH (node)-[r]->(neighbor)
#             RETURN node.name + ' - ' + type(r) + ' -> ' + neighbor.name AS output
#             UNION
#             MATCH (node)<-[r]-(neighbor)
#             RETURN neighbor.name + ' - ' + type(r) + ' -> ' + node.name AS output
#             LIMIT 50
#             """,
#             {"query": "liver cancer"}  # Direct name match
#         )

In [ ]:
# ll = graph.query(
#     '''
#     WITH ["female breast carcinoma"] AS entities
#     MATCH (n)
#     WHERE n.name IN entities
#     OPTIONAL MATCH (n)-[r]->(neighbor)
#     RETURN n.name AS entity, type(r) AS relationship, neighbor.name AS neighbor
#     UNION ALL
#     WITH ["female Breast Carcinoma"] AS entities
#     MATCH (n)
#     WHERE n.name IN entities
#     OPTIONAL MATCH (n)<-[r]-(neighbor)
#     RETURN neighbor.name AS neighbor, type(r) AS relationship, n.name AS entity
#     '''
# )

# ll

In [ ]:
# rel = graph.query(
# """
# MATCH (disease {name: $query, type: 'disease'})
# CALL apoc.path.expandConfig(disease, {
#     relationshipFilter: '>',  // Direction of relationships (outgoing only)
#     maxLevel: 3,              // Maximum number of hops (levels)
#     bfs: true,                // Enforce BFS search order
#     uniqueness: 'NODE_GLOBAL' // Ensure unique nodes at each level
# })
# YIELD path
# RETURN
#     [node IN nodes(path) | node.name] AS path_nodes,
#     [rel IN relationships(path) | type(rel)] AS rel_types
# ORDER BY length(path) ASC  // Order by shortest path first (BFS)

# """,
# {"query": "liver cancer"}

# )





# # rel = graph.query(
# #             """
# #             MATCH (disease {name: $query, type: 'disease'})
# #             CALL apoc.path.expandConfig(disease, {
# #                 relationshipFilter: '>',
# #                 maxLevel: 3,
# #                 limit: 100,
# #                 bfs: true
# #             }) YIELD path
# #             RETURN [node IN nodes(path) | node.name] AS path_nodes,
# #                    [rel IN relationships(path) | type(rel)] AS rel_types
# #             """,
# #             {"query": "liver cancer"}  # Direct name match
# #         )



# # Remove duplicates
# unique_response = []
# seen = set()
# result = ""
# for entry in rel:
#     # Convert the list of nodes and relationships to tuples for comparison
#     entry_tuple = (tuple(entry['path_nodes']), tuple(entry['rel_types']))
#     if entry_tuple not in seen:
#         seen.add(entry_tuple)
#         unique_response.append(entry)

# # Format the output as "entity1 - relationship -> entity2"
# formatted_relations = []
# for entry in unique_response:
#     path_nodes = entry['path_nodes']
#     rel_types = entry['rel_types']

#     if len(rel_types) > 0:
#         for i in range(1, len(path_nodes)):
#             formatted_relations.append(f"{path_nodes[i-1]} - {rel_types[i-1]} -> {path_nodes[i]}")

# result += "\n".join(formatted_relations) + "\n"

In [ ]:
# retriever.get_relevant_documents("Tell me all the genes of liver cancer")

In [ ]:
# def Retriever(question: str):
#     print(f"Search query: {question}")
#     structured_data = structured_retriever(question)
#     unstructured_data = [el.page_content for el in retriever.get_relevant_documents(question)]
#     final_data = f"""Structured data:
#     {structured_data}
#     Unstructured data:
#     {"#Document ". join(unstructured_data)}
#     """
#     return final_data

### The Main Rag Chain

In [ ]:
# # Condense a chat history and follow-up question into a standalone question
# _template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
# in its original language.
# Chat History:
# {chat_history}
# Follow Up Input: {question}
# Standalone question:"""  # noqa: E501
# CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
# from langchain_core.runnables import (
#     RunnableBranch,
#     RunnableLambda,
#     RunnableParallel,
#     RunnablePassthrough,
# )

# from langchain_core.messages import AIMessage, HumanMessage
# from langchain_core.output_parsers import StrOutputParser

In [ ]:
# def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
#     buffer = []
#     for human, ai in chat_history:
#         buffer.append(HumanMessage(content=human))
#         buffer.append(AIMessage(content=ai))
#     return buffer

# _search_query = RunnableBranch(
#     # If input includes chat_history, we condense it with the follow-up question
#     (
#         RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
#             run_name="HasChatHistoryCheck"
#         ),  # Condense follow-up question and chat into a standalone_question

#         RunnablePassthrough.assign(
#             chat_history=lambda x: _format_chat_history(x["chat_history"])
#         )
#         | CONDENSE_QUESTION_PROMPT # formatted chat history and follow-up question to generate a standalone question.
#         | llm2
#         | StrOutputParser(), # parses the LLM output into a string format.
#     ),
#     # Else, we have no chat history, so just pass through the question
#     RunnableLambda(lambda x : x["question"]),
# )

we introduce a prompt that leverages the context provided by the integrated hybrid retriever to produce the response, completing the implementation of the RAG chain.

In [ ]:
# graph.refresh_schema()
# graph.schema

In [ ]:
# prompt = ChatPromptTemplate.from_messages(
#     [
#         # System prompt
#         (
#             "system",
#             """
#             You are an AI assistant tasked with answering questions from the context given below, in the context you will be finding severel relations with nodes and relations, you have identify the right answer, from the given contex. Your responses should be based solely on the context retrieved from the neo4j graph database, ensuring context-rich, precise, and accurate answers without relying on external knowledge or assumptions.

#             Follow these guidelines for both contexts when answering:

#             ### Context-Driven and Accurate Responses:
#             1. Base your answers entirely on the provided context (whether from the graph or vector database). Offer clear, detailed answers if the context provides sufficient information.
#             2. **No Guessing**: If the context lacks an answer, explicitly state that the information is not available. Do not guess or make assumptions based on what isn’t provided.
#             3. **Acknowledge Partial Information**: If the context addresses the question only partially, mention the gaps and provide an answer accordingly.
#             4. **Return All Relevant Information**: If the context contains multiple relevant answers, ensure that you return **all** answers and remove any duplicated entities (like genes, proteins, or drugs) to ensure conciseness.

#             ### Understanding and Interpreting Relationships:
#             5. Recognize key relationships within the knowledge graph:
#                - "Associated with": Implies a 'disease' is connected to a 'gene' or 'protein.'
#                - "interacts_with": Implies a 'gene' is connected to an 'exposure.'
#                - "connected_with": Indicates a connection between a 'gene' and a 'phenotype.'
#                - "drug_enzyme/drug_target/drug_transporter/drug_carrier": Indicates a connection between a 'gene' and a 'drug.'
#                - "drug_side_effect": Represents a connection between 'phenotype' and 'drug' and also vice versa.
#                - "mol_func_interacts": Represents a connection between 'gene' and 'molecular function (mf)'.
#                - "cell_comp_interacts": Represent a connection between 'gene' and 'cellular component (cc)'.
#                - "bioprocess_interacts": Represent a connection between 'gene' and 'Biological_Process (bp)'.
#                - "pathway_interacts": Represent a connection between 'gene' and 'Pathway'.
#                - **No Relationship Found**: If no relationships (e.g., Molecular Function, pathway) are found in the context, clearly state: "Relationship NOT Present."

#             ### Interpret User Intent and Manage Ambiguity:
#             6. Go beyond the literal wording of the query to understand the user's true intent and respond to their needs. In case of ambiguous queries, either ask for clarification or respond to multiple possible interpretations based on the provided context.

#             ### Return Complete and Unique Information:
#             7. Ensure that all relevant entities from both sources (e.g., genes, pathways, drugs) are included in your response, avoiding duplicates. If multiple answers are present, return them all without repetition.

#             ### Clarity and Conciseness:
#             8. Provide answers in a concise, natural tone, avoiding unnecessary repetition or verbose details. Ensure clarity without sacrificing the depth of information.

#             ### Node Finding through transitive relation:
#             9. Understand the user query and from the context, correctly fetch the right answer, deeply try to find the transitive relations very carefully.

#             Context:
#             {context}

#             Question:
#             {question}


#             """
#         ),

#         # Human prompt with instructions and few-shot examples
#         (
#             "human",
#             """
#             You are an AI assistant tasked with answering questions based on data retrieved from a knowledge graph and a vector database, ensuring that responses are clear, context-rich, and precise.

#             ### Examples:

#             **Example 1:**

#             Context:
#             "lung carcinoma - associated_with -> RICTOR\nRICTOR - bioprocess_interacts -> actin cytoskeleton reorganization\nlung carcinoma - associated_with -> RICTOR\nRICTOR - bioprocess_interacts -> regulation of GTPase activity\nlung carcinoma - associated_with -> NTRK3\nNTRK3 - cell_comp_interacts -> glutamatergic synapse\nlung carcinoma - associated_with -> NTRK3\nNTRK3 - cell_comp_interacts -> integral component of postsynaptic membrane"

#             Question:
#             "What genes are involved in lung carcinoma?"

#             Answer:
#             "The key genes involved in lung carcinoma, according to the context, are RICTOR, NTRK3."

#             ---

#             **Example 2:**

#             Context:
#             "lung carcinoma - associated_with -> RICTOR\nRICTOR - bioprocess_interacts -> actin cytoskeleton reorganization\nlung carcinoma - associated_with -> RICTOR\nRICTOR - bioprocess_interacts -> regulation of GTPase activity\nlung carcinoma - associated_with -> NTRK3\nNTRK3 - cell_comp_interacts -> glutamatergic synapse\nlung carcinoma - associated_with -> NTRK3\nNTRK3 - cell_comp_interacts -> integral component of postsynaptic membrane"

#             Question:
#             "What biological processes effected by lung carcinoma?"

#             Answer:
#             "The key biologival processes effected in lung carcinoma, according to the context, are actin cytoskeleton reorganization, regulation of GTPase activity, ."

#             ---

#             **Example 3:**

#             Context:
#             "lung carcinoma - associated_with -> RICTOR\nRICTOR - bioprocess_interacts -> actin cytoskeleton reorganization\nlung carcinoma - associated_with -> RICTOR\nRICTOR - bioprocess_interacts -> regulation of GTPase activity\nlung carcinoma - associated_with -> NTRK3\nNTRK3 - cell_comp_interacts -> glutamatergic synapse\nlung carcinoma - associated_with -> NTRK3\nNTRK3 - cell_comp_interacts -> integral component of postsynaptic membrane"

#             Question:
#             "What cellular components effected by lung carcinoma?"

#             Answer:
#             "The key biologival processes effected in lung carcinoma, according to the context, are  glutamatergic synapse, integral component of postsynaptic membrane ."

#             ---

#             **Example 4:**

#             Context:
#             "The database contains information on several treatments for liver cancer. However, no data is provided regarding alternative therapies."

#             Question:
#             "What therapies are mentioned for liver cancer?"

#             Answer:
#             "Based on the context, no information on alternative therapies for liver cancer is available."

#             ---

#             **Example 5:**

#             Context:
#             "Research highlights the role of protein interactions in disease progression, but no gene-specific data is available."

#             Question:
#             "Which genes are associated with the disease?"

#             Answer:
#             "The context does not provide any gene-specific data for the disease."
#             """
#         ),
#     ]
# )


In [ ]:
# rag_chain = (
#     RunnableParallel(
#         {
#             "context": _search_query | structured_retriever,
#             "question": RunnablePassthrough(),
#         }
#     )
#     | prompt
#     | llm2
#     | StrOutputParser()
# )

# GraphCypherQAChain

In [6]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Gene {name: STRING, source: STRING, type: STRING}
Disease {type: STRING, name: STRING, source: STRING}
Drug {type: STRING, name: STRING, source: STRING}
Anatomy {type: STRING, name: STRING, source: STRING}
Relationship properties:
protein_protein {display_relation: STRING, relation: STRING}
disease_protein {display_relation: STRING, relation: STRING}
drug_protein {display_relation: STRING, relation: STRING}
contraindication {display_relation: STRING, relation: STRING}
indication {display_relation: STRING, relation: STRING}
off_label_use {display_relation: STRING, relation: STRING}
anatomy_protein_present {relation: STRING, display_relation: STRING}
The relationships:
(:Gene)-[:drug_protein]->(:Drug)
(:Gene)-[:protein_protein]->(:Gene)
(:Gene)-[:anatomy_protein_present]->(:Anatomy)
(:Disease)-[:disease_protein]->(:Gene)
(:Disease)-[:contraindication]->(:Drug)
(:Disease)-[:indication]->(:Drug)
(:Disease)-[:off_label_use]->(:Drug)


In [8]:
from pydantic import BaseModel
from langchain_core.pydantic_v1 import Field
from typing import Tuple, List, Optional
from langchain.prompts import ChatPromptTemplate

/home/bernadettem/bernadettenotebook/bernadettem/anaconda3/envs/ritwik_base/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
## ENTITY EXTRACTION

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]"
    )


# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description = "All the name of diseases, drugs, genes or proteins, exposures, molecular_function, pathways, "
        "phenotypes, cellular components, biological processes that appear in the text .",

    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",

            "You are extracting the name of diseases, drugs, genes or proteins, exposures, molecular_function, pathways, "
            "phenotypes, cellular components, biological processes entities from the text ."
            "You are extracting multi-word entities (like the name of pathways or gene names) from the text. "
            "Ensure to capture complete and meaningful phrases, and avoid partial entity identification. "
            "Do semantic searching to understand context and relationships."
            "You need to identify only the names of the entity, not the entity type ."
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm2.with_structured_output(Entities)

In [10]:
from langchain_chroma import Chroma

In [13]:
ques = "Tell me the genes of lung cancer"

entity_chain.invoke(ques)

Entities(names=['lung cancer'])

In [11]:
## VECTOR DATABASE LOADING

vector_db = Chroma(
    embedding_function=embeddings_openai,
    persist_directory="../vector_db",
    collection_name="primekg_new"
)

In [14]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""
    You are an AI language model specializing in information retrieval. Your task is to generate best two different versions of the given user question to
    retrieve relevant documents from a vector database.

    - Your task is to rephrase the user's question in two distinct and meaningful ways, ensuring that the question should be simple and focus solely on the meaning of the
      main question.
    - The two questions should semantically same but they are of differnt form. As a human can ask a same question in many ways.
    - By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.

    Please ensure that each version is unique in phrasing, and explores alternative ways of framing the question while preserving the semantic meaning.


    Do not use any synonyms of certain keyword present in user query. in user query you just need to rephrase, not use different synonyms of any word.

    Lastly return all the differnt phrases.

    Original question: {question}

    1.
    2.
    """
)

In [15]:
from typing import List

from langchain_core.output_parsers import BaseOutputParser
from pydantic import BaseModel, Field


# Output parser will split the LLM result into a list of queries
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines


output_parser = LineListOutputParser()


# Chain
llm_chain = LLMChain(llm=llm2, prompt=QUERY_PROMPT, output_parser=output_parser)

/tmp/ipykernel_28234/1676992513.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm2, prompt=QUERY_PROMPT, output_parser=output_parser)


In [18]:
## MULTI_QUERY_RETRIEVER

retriever = MultiQueryRetriever(
    retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 10}),
    llm_chain = llm_chain,
    parser_key="lines"
)
# pretty_print_docs(doc)

In [19]:
# ! pip install rank_bm25

In [20]:
# ## ENSEMBLE RETRIEVER

# from langchain.retrievers import BM25Retriever, EnsembleRetriever

# bm25_retriever = BM25Retriever.from_documents(docs)
# bm25_retriever.k = 2

# chroma_retriever = vector_db.as_retriever(),

# ensemble_retriever = EnsembleRetriever(
#     retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
# )

In [19]:
# llm3 = ChatGroq (
#     model= "llama-3.1-8b-instant",      # "mixtral-8x7b-32768" #"llama-3.1-70b-versatile", ## "llama-3.1-8b-instant"
#     temperature=0.0,
#     max_tokens=1500,
#     timeout=None,
#     max_retries=4,
#     groq_api_key = api_key

# )

In [19]:
    from langchain_core.prompts.prompt import PromptTemplate
    from langchain.chains import GraphCypherQAChain
    
    
    CYPHER_GENERATION_TEMPLATE = """Task: Generate a Cypher statement to query a graph database.  
    Instructions:  
    Use only the provided relationship types and properties in the schema below.  
    Do not use any other relationship types or properties that are not provided.  
    
    ### Schema Overview:  
    This schema represents a biological knowledge graph containing entities such as diseases, genes, drugs, and anatomy, along with their interactions. Below is the detailed schema:  
     
    
    **Relationships:**  
    - (:Gene)-[:protein_protein]->(:Gene)  
    - (:Gene)-[:drug_protein]->(:Drug)  
    - (:Gene)-[:anatomy_protein_present]->(:Anatomy)  
    - (:Disease)-[:disease_protein]->(:Gene)  
    - (:Disease)-[:contraindication]->(:Drug)  
    - (:Disease)-[:indication]->(:Drug)  
    - (:Disease)-[:off_label_use]->(:Drug)  
    
    ### Additional Guidelines for Query Construction:  
    
    1. **Understanding and Interpreting Relationships:**  
       - Relationships of similar types should be grouped logically for query generation. For example:  
         - `contraindication`, `indication`, and `off_label_use` are all **disease-drug relationships** and should be considered together when querying for disease-drug-related information.  
         - Similarly, any query involving a gene and its interactions should include all relationships linked to the gene, such as `protein_protein`, `drug_protein`, and `disease_protein`.  
    
    2. **Dynamic Query Expansion:**  
       - Ensure that the query dynamically accounts for all relevant relationships and does not hard-code specific connections unless explicitly required.  
    
    3. **Transitive Relation Exploration:**  
       - When a query asks for related entities (e.g., "find all entities connected to a disease"), ensure the query first retrieves the immediate relationships (e.g., disease to gene) and then explores additional layers of connections up to a depth of 3.  
    
    4. **Use of `OPTIONAL MATCH`:**  
       - Use `OPTIONAL MATCH` for querying multiple relationships to ensure that all available data is fetched without strict constraints.  
    
    5. **Data Not Found:**  
       - If the user’s query refers to relationships or entities not present in the schema, respond with: "Information not found."  
    
    6. **General Query Rules:**  
       - Always alias return variables with meaningful names (e.g., `gene`, `disease`, `drug`).  
       - Limit query results to 5000 for efficient execution.  
       - Strive to create syntactically correct Cypher statements and avoid unnecessary characters that might raise errors.  
    
    7. **Clarity and Conciseness:**  
       - Ensure that generated queries are concise yet comprehensive, avoiding verbose constructs.  
    
    8. **Multi-Answer Queries:**  
       - For questions requiring multiple pieces of information, craft a single Cypher query that fetches all relevant data in a unified structure.  
    
    
    """
    
    CYPHER_GENERATION_PROMPT = PromptTemplate(
        input_variables=["question"], template=CYPHER_GENERATION_TEMPLATE
    )
    
    
    cypher_chain = GraphCypherQAChain.from_llm(
        llm = llm2,
        graph=graph,
        verbose=True,
        validate_cypher=True,
        return_intermediate_steps=True,
        top_k = 5000,
        allow_dangerous_requests=True
    )


In [42]:
cypher_chain.invoke("tell me all anatomy of NADK ?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g:Gene {name: 'NADK'})-[:anatomy_protein_present]->(a:Anatomy)
RETURN a

Full Context:
[{'a': {'name': 'saliva-secreting gland', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'Brodmann (1909) area 9', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'omental fat pad', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'anterior cingulate cortex', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'dorsolateral prefrontal cortex', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'adipose tissue of abdominal region', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'layer of synovial tissue', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'squamous epithelium', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'Brodmann (1909) area 46', 'source': 'UBERON', 'type': 'anatomy'}}, {'a': {'name': 'nasal cavity epithelium', 'source': 'UBERON', 'type': 'anatomy'}}

{'query': 'tell me all anatomy of NADK ?',
 'result': "I don't know the answer.",
 'intermediate_steps': [{'query': "cypher\nMATCH (g:Gene {name: 'NADK'})-[:anatomy_protein_present]->(a:Anatomy)\nRETURN a\n"},
  {'context': [{'a': {'name': 'saliva-secreting gland',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'Brodmann (1909) area 9',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'omental fat pad', 'source': 'UBERON', 'type': 'anatomy'}},
    {'a': {'name': 'anterior cingulate cortex',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'dorsolateral prefrontal cortex',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'adipose tissue of abdominal region',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'layer of synovial tissue',
      'source': 'UBERON',
      'type': 'anatomy'}},
    {'a': {'name': 'squamous epithelium',
      'source': 'UBERON',
      'type': 'anat

In [21]:
def retrive_similar(ques):
    l = []
    x_names = []
    retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

    ent = entity_chain.invoke(ques)
    entity = ent.names[0]
    retriever = MultiQueryRetriever(
    retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 10}),
    llm_chain = llm_chain,
    parser_key="lines"
    )
    retrieved_docs = retriever.get_relevant_documents(entity)
    for i in retrieved_docs:
      l.append(i.page_content)

    for entry in l:
        for line in entry.split("\n"):
            if "node_name" in line:
                x_names.append(line.split(":")[1].strip())

    return x_names[0]

# Example use of an LLMChain to rephrase the question
def rephrasing(ques):
    
    sim = retrive_similar(ques)
    ent = entity_chain.invoke(ques)
    entity = ent.names[0]
    # k = []
    # for i in qq:




    qt = ques.replace(entity, sim)

    return qt

In [43]:
ques = "tell me all anatomy of NADK ?"

In [44]:
retrive_similar(ques)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

'Nicotinamide adenine dinucleotide phosphate'

In [45]:
rephrasing(ques)

KeyboardInterrupt: 

In [30]:
# ## PROMPT COMPACT - 02

# from langchain.prompts import ChatPromptTemplate

# prompt_final = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             """
#             "You are an assistant that provides detailed and accurate information based on the provided context given. You must extract only the information given, without adding anything beyond the context."
#             "you should take the unique information from the context provided, and do not deduct or remove any such unique information. "
#             "you try to be focused on the context given, do not go beyond of the context and need not to add any extra thing or explanantion of anything in the answer. "
#             "you need to read the question carefully and after analyzing the question and the context, you need to write the answer. "
#             "You should select all the information provided in the context in a detailed and accurate manner. "
#             "Do not omit, deduct, or add any information beyond the provided context."
#             "The context is provided as a plain text string, and it may include abbreviations like 'g.name' or 'd.name' or 'name' from cypher queries. Understand these abbreviations based on the structure of the cypher query provided."
#             "Do not treat the context as a dictionary or attempt to extract keys from it. Interpret the context as a textual representation and base your responses entirely on the given information."
#             "Your task is to accurately extract relevant details from the provided query and context to answer the question. Focus strictly on what is asked in the question without adding any extra information, assumptions, or explanations."
#             "Ensure you consider all details provided in the context without omitting any unique information."
#             "The query is intended to help you understand the abbreviations or mappings in the context, such as 'g' standing for 'Gene', 'd' for 'Disease', etc."
#             "You are said to treat the context as pure text and don't need to find any key, in the context ."
#             "Always analyze the question in relation to both the query and context before providing the answer."
#             "You should carefully analyze the question, focusing on extracting relevant information based on the provided data. "
#             "In the {query} given, you can understand the {context} and relate the {question} with the {context}, and then answer the question correctly ."
#             "You need understand the things in the serial manner, and answer the question in normal text, with clear format ."
#             "You need to treat the 'query', 'context' as a standalon string, do not treat the 'context' as a dictionary and do not try to find or match any key in the 'context' ."

#             Output only the 'text' in a clear and concise format and in proper human language, do not mention any additional query, context, or question explicitly.

#             Remember, while output the answer, do not mention the user question, query, context in the output, only mention the main 'text' answer .

#             query:
#             {query}

#             context:
#             {context}

#             question:
#             {question}
            

#             """

            
#         ),
#         (
#             "human",
#             """
#             Use the format below to understand the {query} and {context} and answer the {question}. Remember, the context is provided as a plain text string, not as a dictionary, and you should not treat it as such.
#             Only output the 'text' answer, not write the 'question', 'query', 'context' in the output .
#             input: {query}, {context}, {question}

            
#             """
#         )
#     ]
# )


In [23]:
from langchain.prompts import ChatPromptTemplate

prompt_final = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are an assistant providing detailed and accurate information based solely on the provided context.

            Extract and include all relevant details from the context without adding, omitting, or explaining anything beyond the context given.

            Focus on the 'question' and return answer based on the 'query' and 'context' provided.

            When any information is fetched and passed to you by a context, you should return the information, you do not say 'any information is found'.

            Output only the final answer in clear text, without mentioning the query, context, or question.

            You need to output the final answer, in a good phrasing; also you do not need to add any aditional information (e.g. node id, type etc).

            query:
            {query}

            context:
            {context}

            question:
            {question}
            
            """
        ),
        (
            "human",
            """
            Use the format below to understand the {query} and {context} and answer the {question}.
            Only output the 'text' answer, without writing the 'question', 'query', or 'context' in the output.

            input: {query}, {context}, {question}
            """
        )
    ]
)


In [73]:
# l = [{'d': {'name': 'squamous cell carcinoma of colon', 'source': 'MONDO', 'type': 'disease', 'node_id': '18513'}}, {'d': {'name': 'colorectal cancer', 'source': 'MONDO', 'type': 'disease', 'node_id': '5575'}}, {'d': {'name': 'hepatocellular carcinoma', 'source': 'MONDO_grouped', 'type': 'disease', 'node_id': '7256_18902_16216_3243'}}, {'d': {'name': 'lung cancer', 'source': 'MONDO', 'type': 'disease', 'node_id': '8903'}}, {'d': {'name': 'liver cancer', 'source': 'MONDO', 'type': 'disease', 'node_id': '2691'}}, {'d': {'name': 'fibrolamellar hepatocellular carcinoma', 'source': 'MONDO_grouped', 'type': 'disease', 'node_id': '6210_891'}}, {'d': {'name': 'pediatric hepatocellular carcinoma', 'source': 'MONDO', 'type': 'disease', 'node_id': '18055'}}, {'d': {'name': 'adenocarcinoma of liver and intrahepatic biliary tract', 'source': 'MONDO', 'type': 'disease', 'node_id': '18532'}}, {'d': {'name': 'undifferentiated carcinoma of liver and intrahepatic biliary tract', 'source': 'MONDO', 'type': 'disease', 'node_id': '18533'}}]

# l = l
# x = []
# for i in l:
#   x.append(i["d"]["name"])

# x = list(set(x))
# print(x)

In [28]:
## CHAIN TO CONVERT THE MAIN CONTEXT TO HUMAN READABLE FORMAT 

conversion_prompt_template = """
You are given a structured information in a dictionary-like format with important data points. Your task is to convert all the information into a coherent, human-readable paragraph without omitting or altering any details.

Please ensure the following:
- **Use all the information from the 'context_text'**: Do not skip or summarize any details. Every piece of information in the context must be included.
- **Exclude unnecessary details**: Do not include any node identifier (type, id, source etc) in the answer, focus only on the key names and values.
- **Preserve the meaning**: The final output should be accurate, clear, and easy to understand, while retaining the original context's intent.

Here is the context:
{context_text}

Your goal is to present this information clear paragraph format. Ensure every piece of information is represented clearly and concisely.

"""



conversion_prompt = PromptTemplate(
    input_variables=["context_text"],
    template=conversion_prompt_template
)

In [29]:
import time
from langchain.chains import LLMChain


def solution(ques):
    qq = rephrasing(ques)

    
    res = cypher_chain.invoke(qq)
    time.sleep(1)

    if res["result"] != "I don't know the answer.":
        return res["result"]
    
    elif len(res["intermediate_steps"][1]["context"]) == 0:
        return "Sorry, Don't get any information about your question, can you rewrite the question ."

    else:
        # Convert the context (list of dicts) into a plain text format
        context_dict = res["intermediate_steps"][1]["context"]
        context_text = "\n".join([str(item) for item in context_dict])

        # Use LLM to convert the plain context into human-readable form

        conversion_chain = LLMChain(
            llm=llm2,
            prompt=conversion_prompt
        )
        
        human_readable_text = conversion_chain.invoke({
            "context_text": context_text,
        })

        # query = res["intermediate_steps"][0]["query"][7:-1]

        # rag_chain = LLMChain(
        # prompt = prompt_final,
        # llm = llm2
        # )

        # rag_ans = rag_chain.invoke(
        # {
        # "query": query,
        # "context": human_readable_text["text"],  
        # "question": ques
        # }
        # )
        return human_readable_text["text"]  



In [40]:
ques = "tell me the genes of Wilson disease ."

In [41]:
print(solution(ques))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g:Gene)-[:drug_protein]->(d:Drug {name: 'Ferrous gluconate'})
RETURN g

Full Context:
[{'g': {'name': 'FEN1', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'FXN', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'AHSP', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'TFRC', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'NEIL2', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'NEIL1', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'EGLN1', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'HDAC8', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'CP', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'TF', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'FTH1', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'HBA2', 'source': 'NCBI', 'type': 'gene/protein'}}, {'g': {'name': 'POLB', 'source': 

In [59]:
ques = "Tell me all the drugs of triple negative breast cancer."

print(solution(ques))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease {name: 'triple-negative breast carcinoma'})-[:indication]->(dr:Drug)
RETURN dr

Full Context:
[]

> Finished chain.
Sorry, Don't get any information about your question, can you rewrite the question .


In [58]:
ques = "Tell me all the bp and mf associated with triple negative breast cancer ."

print(solution(ques))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease {name: "triple-negative breast carcinoma"})-[:associated_with]->(g:Gene)-[:bioprocess_interacts]->(bp:Biological_Process), 
      (g)-[:mol_func_interacts]->(mf:Molecular_Function)
RETURN bp, mf

Full Context:
[{'bp': {'name': 'negative regulation of neuron apoptotic process', 'source': 'GO', 'type': 'biological_process', 'node_id': 43524}, 'mf': {'name': 'protein binding', 'source': 'GO', 'type': 'molecular_function', 'node_id': 5515}}, {'bp': {'name': 'response to glucocorticoid', 'source': 'GO', 'type': 'biological_process', 'node_id': 51384}, 'mf': {'name': 'protein binding', 'source': 'GO', 'type': 'molecular_function', 'node_id': 5515}}, {'bp': {'name': 'actin cytoskeleton organization', 'source': 'GO', 'type': 'biological_process', 'node_id': 30036}, 'mf': {'name': 'protein binding', 'source': 'GO', 'type': 'molecular_function', 'node_id': 5515}}, {'bp': {'name': 'positive regulation of protei

In [70]:
ques = "Tell me all genes of triple negative breast cancer ."

solution(ques)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [28]:
ques = "Tell me all the genes and pathways of skin cancer ."

solution(ques)

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease {name: 'skin cancer'})-[:associated_with]->(g:Gene)-[:pathway_interacts]->(p:Pathway)
RETURN g, p

Full Context:
[{'g': {'name': 'MMP9', 'source': 'NCBI', 'type': 'gene/protein', 'node_id': 4318}, 'p': {'name': 'Signaling by SCF-KIT', 'source': 'REACTOME', 'type': 'pathway', 'node_id': 'R-HSA-1433557'}}, {'g': {'name': 'MMP9', 'source': 'NCBI', 'type': 'gene/protein', 'node_id': 4318}, 'p': {'name': 'Collagen degradation', 'source': 'REACTOME', 'type': 'pathway', 'node_id': 'R-HSA-1442490'}}, {'g': {'name': 'MMP9', 'source': 'NCBI', 'type': 'gene/protein', 'node_id': 4318}, 'p': {'name': 'Degradation of the extracellular matrix', 'source': 'REACTOME', 'type': 'pathway', 'node_id': 'R-HSA-1474228'}}, {'g': {'name': 'MMP9', 'source': 'NCBI', 'type': 'gene/protein', 'node_id': 4318}, 'p': {'name': 'EPH-ephrin mediated repulsion of cells', 'source': 'REACTOME', 'type': 'pathway', 'node_id': 'R-HSA-392866

KeyboardInterrupt: 

In [41]:
ques = "Tell me all the name of bp associated with liver cancer ."


qq = rephrasing(ques)

res = cypher_chain.invoke(qq)

context_dict = res["intermediate_steps"][1]["context"]
context_text = "\n".join([str(item) for item in context_dict])

# Use LLM to convert the plain context into human-readable form

conversion_chain = LLMChain(
    llm=llm2,
    prompt=conversion_prompt
)

human_readable_text = conversion_chain.invoke({
    "context_text": context_text,
})

print(human_readable_text["text"])

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease {name: 'liver cancer'})-[:associated_with]->(:Gene)-[:bioprocess_interacts]->(bp:Biological_Process)
RETURN bp.name

Full Context:
[{'bp.name': 'platelet degranulation'}, {'bp.name': 'negative regulation of endopeptidase activity'}, {'bp.name': 'stem cell differentiation'}, {'bp.name': 'regulation of small GTPase mediated signal transduction'}, {'bp.name': 'extracellular matrix disassembly'}, {'bp.name': 'negative regulation of complement activation, lectin pathway'}, {'bp.name': 'negative regulation of blood coagulation, intrinsic pathway'}, {'bp.name': 'regulation of cell cycle'}, {'bp.name': 'cellular response to DNA damage stimulus'}, {'bp.name': 'positive regulation of apoptotic process'}, {'bp.name': 'cell adhesion'}, {'bp.name': 'positive regulation of cold-induced thermogenesis'}, {'bp.name': 'nervous system development'}, {'bp.name': 'protein deubiquitination'}, {'bp.name': 'positive regulat

In [105]:
ques = "What are the biological processes and mol function present in pediatric hepatocellular carcinoma ?"
solution(ques)

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease {name: 'pediatric hepatocellular carcinoma'})-[:associated_with]->(g:Gene)-[:bioprocess_interacts]->(bp:Biological_Process), 
      (g)-[:mol_func_interacts]->(mf:Molecular_Function) 
RETURN DISTINCT bp.name AS Metabolic_Process, mf.name AS Molecular_Function

Full Context:
[{'Metabolic_Process': 'platelet degranulation', 'Molecular_Function': 'interleukin-1 binding'}, {'Metabolic_Process': 'negative regulation of endopeptidase activity', 'Molecular_Function': 'interleukin-1 binding'}, {'Metabolic_Process': 'stem cell differentiation', 'Molecular_Function': 'interleukin-1 binding'}, {'Metabolic_Process': 'regulation of small GTPase mediated signal transduction', 'Molecular_Function': 'interleukin-1 binding'}, {'Metabolic_Process': 'extracellular matrix disassembly', 'Molecular_Function': 'interleukin-1 binding'}, {'Metabolic_Process': 'negative regulation of complement activation, lectin pathway', 'M

'The biological processes associated with pediatric hepatocellular carcinoma include platelet degranulation, negative regulation of endopeptidase activity, stem cell differentiation, regulation of small GTPase mediated signal transduction, extracellular matrix disassembly, negative regulation of complement activation (lectin pathway), negative regulation of blood coagulation (intrinsic pathway), Wnt signaling pathway, and insulin receptor signaling pathway. \n\nThe molecular functions related to these processes encompass interleukin-1 binding, endopeptidase inhibitor activity, enzyme binding, tumor necrosis factor binding, protease binding, growth factor binding, interleukin-8 binding, serine-type endopeptidase inhibitor activity, signaling receptor binding, calcium-dependent protein binding, protein binding, protein kinase binding, ubiquitin protein ligase binding, gamma-catenin binding, microtubule plus-end binding, dynein complex binding, protein kinase regulator activity, transcrip

In [69]:
ques = "What are the name of the genes that has no pathways that associated with liver cancer ?"
solution(ques)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [66]:
def split_context_into_batches(context_text, max_batch_size):
    context_lines = context_text.split("\n")
    for i in range(0, len(context_lines), max_batch_size):
        yield "\n".join(context_lines[i:i + max_batch_size])

In [67]:
ques = "Tell me all the pathways associated with familial prostate carcinoma ."

res = cypher_chain.invoke(ques)

context_dict = res["intermediate_steps"][1]["context"]
context_text = "\n".join([str(item) for item in context_dict])

context_text



> Entering new GraphCypherQAChain chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("prime_kg_files/pkg_drug_phenotype.csv")
df

,Unnamed: 0,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
0,3348335,drug_effect,side effect,16322,DB00583,drug,Levocarnitine,DrugBank,23158,2027,effect/phenotype,Abdominal pain,HPO
1,3348336,drug_effect,side effect,16322,DB00583,drug,Levocarnitine,DrugBank,85849,4396,effect/phenotype,Poor appetite,HPO
2,3348337,drug_effect,side effect,16322,DB00583,drug,Levocarnitine,DrugBank,22447,739,effect/phenotype,Anxiety,HPO
3,3348338,drug_effect,side effect,16322,DB00583,drug,Levocarnitine,DrugBank,22831,11675,effect/phenotype,Arrhythmia,HPO
4,3348339,drug_effect,side effect,16322,DB00583,drug,Levocarnitine,DrugBank,23469,3418,effect/phenotype,Back pain,HPO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58917,3413114,drug_effect,side effect,15288,DB00105,drug,Interferon alfa-2b,DrugBank,25235,2595,effect/phenotype,Ileus,HPO
58918,3413115,drug_effect,side effect,15288,DB00105,drug,Interferon alfa-2b,DrugBank,24289,1618,effect/phenotype,Dysphonia,HPO
58919,3413116,drug_effect,side effect,15288,DB00105,drug,Interferon alfa-2b,DrugBank,23834,1677,effect/phenotype,Coronary artery atherosclerosis,HPO
58920,3413117,drug_effect,side effect,15288,DB00105,drug,Interferon alfa-2b,DrugBank,84853,1262,effect/phenotype,Excessive daytime somnolence,HPO
